In [1]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SparkSession

Deskrispi Data :

    CustomerName : Nama 
    Age :   umur
    MaritalStatus : (0) =  Belum Menikah 
                    (1) = Sudah Menikah
    IncomeRange : ---
    Gender  : (0) = Laki- Laki 
            (1) = Perempuan
    TotalChildren:  ---
    ChildrenAtHome : ---
    Education : (0) = SMA
                (1) = S1
                (2) = Post Graduate
                (3) = S2
                (4) =  S3
    Occupation :    (0) = sales
                    (1) = skilled manual work
                    (2) = unskilled manual work
                    (3) = professional
                    (4) = clerical work 
                    (5) = management
    HomeOwner : (0) = tidak memiliki  rumah  sendiri
                (1) = memiliki rumah sendiri
    Cars  :   ---


In [2]:
appName = "Klastering di Spark"
spark = SparkSession \
    .builder \
    .appName(appName) \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [3]:
customers = spark.read.csv(
    'D:\SEKOLAH\KULIAH\SEMESTER 6\PENAMBANGAN DATA\Clustering K-Means\clean_customer_data_6.csv', inferSchema=True, header=True)
customers.show(2)

+---+----------------+---+-------------+-----------+------+-------------+--------------+---------+----------+---------+----+
|_c0|    CustomerName|Age|MaritalStatus|IncomeRange|Gender|TotalChildren|ChildrenAtHome|Education|Occupation|HomeOwner|Cars|
+---+----------------+---+-------------+-----------+------+-------------+--------------+---------+----------+---------+----+
|  0|      Wati Putra| 69|            0|     110527|     1|            4|             4|        4|         2|        0|   0|
|  1|Cahyadi Haryanto| 63|            1|     125609|     0|            0|             5|        1|         1|        1|   0|
+---+----------------+---+-------------+-----------+------+-------------+--------------+---------+----------+---------+----+
only showing top 2 rows



In [4]:
assembler = VectorAssembler(inputCols = [
    "Age", "MaritalStatus", "IncomeRange", "Gender", "TotalChildren",
    "ChildrenAtHome", "Education", "Occupation", "HomeOwner", "Cars"],
                            outputCol="features")
train = assembler.transform(customers).select('CustomerName', 'features')
train.show(truncate = False, n=2)

+----------------+-----------------------------------------------+
|CustomerName    |features                                       |
+----------------+-----------------------------------------------+
|Wati Putra      |[69.0,0.0,110527.0,1.0,4.0,4.0,4.0,2.0,0.0,0.0]|
|Cahyadi Haryanto|[63.0,1.0,125609.0,0.0,0.0,5.0,1.0,1.0,1.0,0.0]|
+----------------+-----------------------------------------------+
only showing top 2 rows



In [5]:
kmeans = KMeans(
    featuresCol=assembler.getOutputCol(), predictionCol="cluster",
    k=5, seed=0)
model = kmeans.fit(train)
print ("Model selesai dibuat!")

Model selesai dibuat!


In [6]:
centers = model.clusterCenters() 
print("Cluster Centers: ") 
for center in centers: 
    print(center)

Cluster Centers: 
[4.38759058e+01 5.08152174e-01 1.36054785e+05 5.01358696e-01
 2.51675725e+00 1.25860507e+00 1.98958333e+00 2.50815217e+00
 4.81431159e-01 1.52898551e+00]
[4.38693416e+01 5.01543210e-01 3.26506199e+04 5.20061728e-01
 2.51388889e+00 1.23713992e+00 1.97067901e+00 2.50102881e+00
 4.95884774e-01 1.51646091e+00]
[4.31859244e+01 5.10504202e-01 8.40943634e+04 4.94747899e-01
 2.45535714e+00 1.28151261e+00 2.00157563e+00 2.49002101e+00
 5.01050420e-01 1.46638655e+00]
[4.31295732e+01 4.98475610e-01 5.81993582e+04 4.90853659e-01
 2.49491870e+00 1.25965447e+00 1.96138211e+00 2.42225610e+00
 4.88313008e-01 1.48780488e+00]
[4.30288462e+01 5.05060729e-01 1.09898108e+05 4.92914980e-01
 2.52732794e+00 1.22115385e+00 1.97975709e+00 2.45242915e+00
 5.12145749e-01 1.48937247e+00]


In [7]:
prediction = model.transform(train)#melakukan prediksi klaster
prediction.groupBy("cluster").count().orderBy("cluster").show()
prediction.select('CustomerName', 'cluster').show()

+-------+-----+
|cluster|count|
+-------+-----+
|      0| 2201|
|      1| 1952|
|      2| 1910|
|      3| 1968|
|      4| 1969|
+-------+-----+

+----------------+-------+
|    CustomerName|cluster|
+----------------+-------+
|      Wati Putra|      4|
|Cahyadi Haryanto|      0|
|    Maya Pratama|      1|
|    Zainab Hakim|      2|
|    Wati Pratama|      1|
|     Lina Fauzan|      1|
|    Maya Rachman|      4|
|   Siti Muliawan|      1|
|Zainab Anggraini|      3|
|   Fitri Santoso|      3|
|Cahyadi Haryanto|      3|
|    Budi Santoso|      0|
|  Wati Anggraini|      1|
|    Joko Purnomo|      0|
|    Indra Fauzan|      3|
|    Dewi Saputra|      0|
|      Lina Utomo|      1|
|     Eko Santoso|      1|
|   Maya Setiawan|      1|
|    Rizki Fauzan|      0|
+----------------+-------+
only showing top 20 rows



In [8]:
prediction.show()

+----------------+--------------------+-------+
|    CustomerName|            features|cluster|
+----------------+--------------------+-------+
|      Wati Putra|[69.0,0.0,110527....|      4|
|Cahyadi Haryanto|[63.0,1.0,125609....|      0|
|    Maya Pratama|[22.0,0.0,43943.0...|      1|
|    Zainab Hakim|[69.0,1.0,85893.0...|      2|
|    Wati Pratama|[44.0,0.0,43524.0...|      1|
|     Lina Fauzan|[24.0,1.0,24980.0...|      1|
|    Maya Rachman|[68.0,1.0,111260....|      4|
|   Siti Muliawan|[63.0,0.0,44985.0...|      1|
|Zainab Anggraini|[33.0,1.0,46491.0...|      3|
|   Fitri Santoso|[57.0,0.0,55478.0...|      3|
|Cahyadi Haryanto|[20.0,1.0,50115.0...|      3|
|    Budi Santoso|[28.0,0.0,126194....|      0|
|  Wati Anggraini|[67.0,0.0,34471.0...|      1|
|    Joko Purnomo|[48.0,0.0,147860....|      0|
|    Indra Fauzan|[38.0,0.0,66408.0...|      3|
|    Dewi Saputra|[62.0,1.0,136252....|      0|
|      Lina Utomo|[42.0,1.0,26724.0...|      1|
|     Eko Santoso|[27.0,1.0,21578.0...| 

In [10]:
new_df = prediction['features','cluster']

In [11]:
new_df.show()

+--------------------+-------+
|            features|cluster|
+--------------------+-------+
|[69.0,0.0,110527....|      4|
|[63.0,1.0,125609....|      0|
|[22.0,0.0,43943.0...|      1|
|[69.0,1.0,85893.0...|      2|
|[44.0,0.0,43524.0...|      1|
|[24.0,1.0,24980.0...|      1|
|[68.0,1.0,111260....|      4|
|[63.0,0.0,44985.0...|      1|
|[33.0,1.0,46491.0...|      3|
|[57.0,0.0,55478.0...|      3|
|[20.0,1.0,50115.0...|      3|
|[28.0,0.0,126194....|      0|
|[67.0,0.0,34471.0...|      1|
|[48.0,0.0,147860....|      0|
|[38.0,0.0,66408.0...|      3|
|[62.0,1.0,136252....|      0|
|[42.0,1.0,26724.0...|      1|
|[27.0,1.0,21578.0...|      1|
|[28.0,1.0,27239.0...|      1|
|[69.0,1.0,131249....|      0|
+--------------------+-------+
only showing top 20 rows



In [13]:
train_df,test_df = new_df.randomSplit([0.8,0.2],seed =42)

In [33]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.classification import LinearSVC

rf = RandomForestClassifier(labelCol="cluster", featuresCol="features", numTrees=10)
nv = NaiveBayes(labelCol='cluster',featuresCol='features' )

# Train the model
model = rf.fit(train_df)
model_nv = nv.fit(train_df)



In [25]:
predictions = model.transform(test_df)
predictions_nv = model_nv.transform(test_df)
predictions.select("features", "cluster", "prediction").show()
predictions_nv.select("features", "cluster", "prediction").show()

+--------------------+-------+----------+
|            features|cluster|prediction|
+--------------------+-------+----------+
|(10,[0,1,2,3,4],[...|      1|       1.0|
|(10,[0,1,2,3,7],[...|      0|       0.0|
|(10,[0,1,2,3,8],[...|      0|       0.0|
|(10,[0,1,2,4,5],[...|      3|       3.0|
|(10,[0,1,2,4,6],[...|      4|       4.0|
|(10,[0,1,2,4,6],[...|      0|       0.0|
|(10,[0,1,2,4,7],[...|      3|       3.0|
|(10,[0,1,2,4,7],[...|      3|       3.0|
|(10,[0,1,2,4,9],[...|      1|       1.0|
|(10,[0,1,2,4,9],[...|      0|       0.0|
|(10,[0,1,2,4,9],[...|      4|       4.0|
|(10,[0,1,2,4,9],[...|      0|       0.0|
|(10,[0,1,2,4,9],[...|      4|       4.0|
|(10,[0,1,2,5,6],[...|      0|       0.0|
|(10,[0,1,2,5,7],[...|      1|       1.0|
|(10,[0,1,2,5,9],[...|      4|       4.0|
|(10,[0,1,2,6,7],[...|      4|       4.0|
|(10,[0,1,2,7,9],[...|      2|       2.0|
|(10,[0,1,2,8,9],[...|      4|       4.0|
|(10,[0,2,3],[66.0...|      3|       3.0|
+--------------------+-------+----

In [24]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Initialize the evaluator
evaluator = MulticlassClassificationEvaluator(labelCol="cluster", predictionCol="prediction", metricName="accuracy")

# Evaluate the accuracy
accuracy = evaluator.evaluate(predictions)
accuracy_nv = evaluator.evaluate(predictions_nv)
print(f"Test Accuracy random forest : {accuracy}")
print(f"Test Accuracy Naive Bayes : {accuracy_nv}")

Test Accuracy random forest : 0.9448204060385216
Test Accuracy Naive Bayes : 0.5111920874544508
